<a href="https://colab.research.google.com/github/rtrochepy/astronomer/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm
!pip install imbalanced-learn

In [33]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, balanced_accuracy_score
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler

# 1. Procesamiento de Fechas
def process_dates(df, date_columns):
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
            df[col] = (df[col] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1d')
    return df

# 2. Manejo de Valores Nulos
def handle_missing_values(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    cat_cols = df.select_dtypes(include=[object, 'category']).columns

    # Imputar numéricas con la media
    imputer_num = SimpleImputer(strategy='mean')
    df[numeric_cols] = imputer_num.fit_transform(df[numeric_cols])

    # Imputar categóricas con "Missing"
    for col in cat_cols:
        df[col] = df[col].fillna("Missing")
    return df

# 3. Procesamiento de Variables Categóricas
def process_categorical(df):
    cat_cols = df.select_dtypes(include=[object, 'category']).columns
    for col in cat_cols:
        df[col] = LabelEncoder().fit_transform(df[col].astype(str))
    return df

# 4. Balanceo con ADASYN y chunks
def balance_data_with_chunks(X, y, chunk_size=1000):
    sampler = ADASYN(random_state=42)
    fallback_sampler = RandomUnderSampler(random_state=42)
    balanced_X, balanced_y = [], []

    for i in range(0, len(X), chunk_size):
        X_chunk, y_chunk = X[i:i + chunk_size], y[i:i + chunk_size]
        class_dist = dict(pd.Series(y_chunk).value_counts())
        print(f"Chunk {i}-{i+chunk_size}: Distribución de clases: {class_dist}")

        if len(class_dist) < 2:
            print(f"Chunk {i}-{i+chunk_size} omitido: solo contiene una clase.")
            continue

        try:
            X_res, y_res = sampler.fit_resample(X_chunk, y_chunk)
        except ValueError:
            print(f"ADASYN falló en el chunk {i}-{i+chunk_size}, aplicando undersampling.")
            X_res, y_res = fallback_sampler.fit_resample(X_chunk, y_chunk)

        balanced_X.append(X_res)
        balanced_y.append(y_res)

    if not balanced_X:
        raise ValueError("No se pudo balancear ningún chunk.")
    return np.vstack(balanced_X), np.hstack(balanced_y)

# Flujo principal
file_path = "data_labels.csv"  # Ruta del archivo CSV
date_columns = ['Expenditure_AHF']
label_column = 'label'

print("Cargando y preprocesando los datos...")
df = pd.read_csv(file_path)

# Procesamiento
df = process_dates(df, date_columns)
df = handle_missing_values(df)
df = process_categorical(df)

# Verificación final
print("Verificando que todos los datos sean numéricos...")
X = df.drop(columns=[label_column])
y = df[label_column]

# Asegurar conversión a numérico
X = X.apply(pd.to_numeric, errors='coerce')

if X.isnull().any().any():
    raise ValueError("Persisten valores no numéricos o NaN en los datos.")

# Balancear datos
print("Balanceando los datos con ADASYN...")
try:
    X_balanced, y_balanced = balance_data_with_chunks(X.values, y.values, chunk_size=1000)
except ValueError as e:
    print(f"Error crítico: {e}. Intentando con chunks más pequeños...")
    X_balanced, y_balanced = balance_data_with_chunks(X.values, y.values, chunk_size=500)

# División en Entrenamiento y Prueba
print("Dividiendo en entrenamiento y prueba...")
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

# Estandarización
print("Estandarizando los datos...")
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Selección de Características
print("Seleccionando características con SelectKBest...")
selector = SelectKBest(score_func=f_classif, k=20)
X_train_sel = selector.fit_transform(X_train, y_train)
X_test_sel = selector.transform(X_test)

# Modelo y Evaluación
print("Entrenando el modelo...")
model = RandomForestClassifier(random_state=42)
model.fit(X_train_sel, y_train)
y_pred = model.predict(X_test_sel)

metrics = {
    'ROC AUC': roc_auc_score(y_test, model.predict_proba(X_test_sel)[:, 1]),
    'F1 Score': f1_score(y_test, y_pred),
    'Balanced Accuracy': balanced_accuracy_score(y_test, y_pred)
}
print("Métricas del modelo:", metrics)

# Guardar resultados
final_df = pd.DataFrame(X_train_sel, columns=[f"Feature_{i}" for i in range(X_train_sel.shape[1])])
final_df['label'] = y_train
final_df.to_csv("processed_dataset.csv", index=False)
print("Dataset procesado guardado como 'processed_dataset.csv'.")

Cargando y preprocesando los datos...
Verificando que todos los datos sean numéricos...
Balanceando los datos con ADASYN...
Chunk 0-1000: Distribución de clases: {0.0: 1000}
Chunk 0-1000 omitido: solo contiene una clase.
Chunk 1000-2000: Distribución de clases: {0.0: 1000}
Chunk 1000-2000 omitido: solo contiene una clase.
Chunk 2000-3000: Distribución de clases: {0.0: 1000}
Chunk 2000-3000 omitido: solo contiene una clase.
Chunk 3000-4000: Distribución de clases: {0.0: 1000}
Chunk 3000-4000 omitido: solo contiene una clase.
Chunk 4000-5000: Distribución de clases: {0.0: 1000}
Chunk 4000-5000 omitido: solo contiene una clase.
Chunk 5000-6000: Distribución de clases: {0.0: 1000}
Chunk 5000-6000 omitido: solo contiene una clase.
Chunk 6000-7000: Distribución de clases: {0.0: 1000}
Chunk 6000-7000 omitido: solo contiene una clase.
Chunk 7000-8000: Distribución de clases: {0.0: 1000}
Chunk 7000-8000 omitido: solo contiene una clase.
Chunk 8000-9000: Distribución de clases: {0.0: 1000}
Chunk

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 10  74  78  87 104 108 111 146 148 149 150 155 157 172] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Métricas del modelo: {'ROC AUC': 0.5, 'F1 Score': 0.5, 'Balanced Accuracy': 0.25}
Dataset procesado guardado como 'processed_dataset.csv'.
